<p style="text-align:center">
    <a href="https://www.ict.mahidol.ac.th/en/" target="_blank">
    <img src="https://www3.ict.mahidol.ac.th/ICTSurveysV2/Content/image/MUICT2.png" width="400" alt="Faculty of ICT">
    </a>
</p>

# Lab07: ML Basics: Regression

This Lab Assignment is an exercise in Regression modelling using a more sophisticated model to predict and understand house prices in the Melbourne dataset (2017). The assignment will guide you to train a more advanced model on the pricing dataset, report and compare its error scores, and view its learning curves. Furthermore, the assignment will take you through steps to export the model and make predictions within a web dashboard.

__Learning Objectives are:__
* Recognize Overfitting & Underfitting.
* Compare Models with Error Metrics (MAE, R2)
* Deliver Predictions in a Dashboard
* Inform users that model predictions come with errors: **f(x) = y_pred + error + noise**


__Instructions:__
1. Append your ID at the end of this jupyter file name. For example, ```ITCS227_Lab07_Assignment_6788123.ipynb```
2. Complete each task in the lab.
3. Once finished, raise your hand to call a TA.
4. The TA will check your work and give you an appropriate score.
5. Submit your IPYNB source code to MyCourse as record-keeping.

## 0. Install Packages:
- Uncomment this line to install the `Streamlit` dashboard library, we will use later.

*If your machine reports other missing libraries, then you may need to install the library using pip. (Mostly the libraries used are already installed with Anaconda).*

In [ ]:
#!pip install streamlit

# Explaining Melbourne Housing Market - 

### Dataset Source: 

Melbourne real estate is BOOMING. Can you find the insight or predict the next big trend to become a real estate mogul… or even harder, to snap up a reasonably priced 2-bedroom unit?

**Content**
- This is a snapshot of a dataset created by Tony Pino.
- It was scraped from publicly available results posted every week from Domain.com.au. He cleaned it well, and now it's up to you to make data analysis magic. The dataset includes Address, Type of Real estate, Suburb, Method of Selling, Rooms, Price, Real Estate Agent, Date of Sale and distance from C.B.D.

**Notes on Specific Variables**
- Rooms: Number of rooms
- Price: Price in dollars
- Method: S - property sold; SP - property sold prior; PI - property passed in; PN - sold prior not disclosed; SN - sold not disclosed; NB - no bid; VB - vendor bid; W - withdrawn prior to auction; SA - sold after auction; SS - sold after auction price not disclosed. N/A - price or highest bid not available.
- Type: br - bedroom(s); h - house,cottage,villa, semi,terrace; u - unit, duplex; t - townhouse; dev site - development site; o res - other residential.
- SellerG: Real Estate Agent
- Date: Date sold
- Distance: Distance from CBD
- Regionname: General Region (West, North West, North, North east …etc)
- Propertycount: Number of properties that exist in the suburb.
- Bedroom2 : Scraped # of Bedrooms (from different source)
- Bathroom: Number of Bathrooms
- Car: Number of carspots
- Landsize: Land Size
- BuildingArea: Building Size
- CouncilArea: Governing council for the area

**Acknowledgements**
- This is intended as a static (unchanging) snapshot of https://www.kaggle.com/anthonypino/melbourne-housing-market. It was created in September 2017. Additionally, homes with no Price have been removed.

### Load Dataset:

In [ ]:
import pandas as pd

df = pd.read_csv('files/melb_data.csv').drop('Unnamed: 0',axis=1)

### Visualize the Dataset:

In [ ]:
import plotly.express as px
import plotly.io as pio

def plotly_map(df, latlng_cols=('lat','lng'), z=None, custom_data_cols=[], custom_text=[], center_dict=dict(lat=13.6, lon=100.4), zoom=5, WRITE=False, WRITE_FN=None):
    """ 
    @WRITE_FN - do not include extension - i.e. `.png` or `.html`, as both files will be written.
    Docs:   https://plotly.com/python-api-reference/generated/plotly.express.density_mapbox.html
            https://plotly.com/python/mapbox-density-heatmaps/
    """
    pio.templates.default = 'plotly_white' # 'plotly_dark'
    fig = px.density_mapbox(df, 
                            lat=latlng_cols[0], 
                            lon=latlng_cols[1], 
                            z=z,
                            radius=5,
                            center=center_dict, zoom=zoom,
                            mapbox_style=["open-street-map",'carto-darkmatter'][0],
                            custom_data=custom_data_cols,
                           )

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    if custom_text:
        fig.update_traces(
            hovertemplate="<br>".join(custom_text)
        )
    fig.show(config={'displayModeBar': False} )
    if WRITE:
        if WRITE_FN != None and isinstance(WRITE_FN, str) and len(WRITE_FN)>4:
            ofn = f'{WRITE_FN}_MapPlot_{TIMESTAMP_FILENAME()}'
            fig.write_image(ofn+'.png')
            fig.write_html(ofn+'.html', full_html=False, include_plotlyjs=False, include_mathjax=False )

In [ ]:
dfm = df.dropna()
plotly_map(dfm[dfm['Method']=='S'], 
           latlng_cols=('Lattitude','Longtitude'), 
           z='Price',
           custom_data_cols=['CouncilArea',
                             'Distance',
                             'Landsize',
                             'BuildingArea',
                             'Rooms',
                             'Bathroom',
                             'Price'
                           ], 
           custom_text=['Area: %{customdata[0]}',
                        'Distance: %{customdata[1]}',
                        'LS / BA: %{customdata[2]}/%{customdata[3]}',
                        'Rm / Br: %{customdata[4]}/%{customdata[5]}',
                        'Price AUD-$: %{customdata[6]:.,1f}'
                            ],
           center_dict=dict(lat=-37.814, lon=144.963),
           zoom=9
          )

### Display Dataset Features:

In [ ]:
print('Total Num of Records ',len(df))
display( df.dtypes )
display( df.head(2) )
display( df.describe().T )

## Collect the Dataset:

In [ ]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

def get_some_data(data, features, target='Price'):
    _data = data.dropna()
    X = _data[features]
    y = _data[target]
    return X, y

# Choose Features, Model & Measure Errors:

## Q1: - Choose the Input Features (Columns) for `X` from the `df` Dataset to Predict House Prices `Y`:

* To learn the model, we need define the `X` and the `Y`.
* For `X` - we started with `['Distance']`.
* You can choose from your `df` (DataFrame above).
* For `Y` - use the `Price` (as we did earlier).

In [ ]:
# insert your code here
features = [ ]
# X, y = get_some_data( df, features )
print('Number of Features: ', len(features))
print('Features Names: ', features)

* If you've selected `Categorical` features, uncomment the following to convert them to numeric.

In [ ]:
# ------------ Uncomment this section if you choose to use Categorical / String / Object features:
# ------------ This will encode the Strings as Integers.
# from sklearn.preprocessing import LabelEncoder
# import json
# 
# def export_mapping_to_json(mapping:dict):
#     def convert_numpy(obj): # Recursive function to handle nested structures
#         if isinstance(obj, np.integer): return int(obj)
#         elif isinstance(obj, np.floating): return float(obj)
#         elif isinstance(obj, np.ndarray): return obj.tolist()
#         elif isinstance(obj, dict): return {k: convert_numpy(v) for k, v in obj.items()}
#         elif isinstance(obj, list) or isinstance(obj, tuple): return [convert_numpy(elem) for elem in obj]
#         else: return obj
#     ofn = 'x_categorical_to_integer_mapping.json'
#     with open(ofn, 'w') as f:
#         json.dump(convert_numpy(mapping), f, indent=4)
#         print(f"Mapping successfully exported to {ofn}")
# 
# def encode_object_to_integers( X ):
#     mapping = {}
#     le = LabelEncoder()
#     for col in X.select_dtypes(include=['object']).columns:
#         X[col] = le.fit_transform(X[col])
#         mapping[col] = dict(zip(le.classes_, le.transform(le.classes_)))
#     export_mapping_to_json(mapping)
#
# encode_object_to_integers( X )
# ------------ 

## Q2: - Split the X and Y into Train and Test sets:

In [ ]:
from sklearn.model_selection import train_test_split
# insert your code here




<details><summary><span style="color:red">&#x1F6C8; Help</span> (Use this only as a last resort!!)</summary>

```Python
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0) # Split into 85% training and 15% testing data
```

</details>

## Q3: - Choose a Regression Model Algorithm to Fit:

1. Find a regression model to try out.
2. Add the import statement for the model.
3. Instantiate the model.
4. Call the `fit( ... )` function on the model, as we did in Tutorials 06 & 07.

In [ ]:
from sklearn.linear_model import * 
from sklearn.ensemble import *
# insert your code here



<details><summary><span style="color:#17c1e5">&#x1F6C8; Hint</span> (Extra info)</summary>

* You can take a read of the [Scikit-Learn Documentation](https://scikit-learn.org/stable/modules/ensemble.html) which have several good choices of `regression models` for tabular datasets (both numeric and categorical features).
* You might ask Mahidol's Gemini Account to give you some ideas. It has to be a `regression model` to predict a single number.

</details>

<details><summary><span style="color:red">&#x1F6C8; Help</span> (Use this only as a last resort!!)</summary>

* We can use the `HistGradientBoostingRegressor` which is an excellent model for tabular data, for regression (and for classification using `HistGradientBoostingClassifier`).
> * This model builds an "[ensemble](https://scikit-learn.org/stable/modules/ensemble.html)" of `100` "decision trees" (a flowchart) and iteratively improves its decisions (accuracy) to reduce its errors.
> * The collection of all trees gives the most balanced, least error result, which is a weighted combination of all trees.
> * Like all Ensembles, GBR is designed to manage error improvements on both train and test sets.
> 
> **Reference:**  Friedman, J.H. (2001). Greedy function approximation: A gradient boosting machine. Annals of Statistics, 29, 1189-1232.

* There are more recent and more advanced models. This one is simple to use and gives good results. It will automatically encode categorical items (similar to one-hot encoding), and handles numeric value as we will see.

```python
my_model = HistGradientBoostingRegressor()
my_model.fit( X_train, y_train )
```
* or you can try:
```python
my_model = RandomForestRegressor()
my_model.fit( X_train, y_train )
```
* or you can try:
```python
my_model = LinearRegression()
my_model.fit( X_train, y_train )

```

</details>

## Q4: - Measure the MAE and R2 errors for the `test` set of this new model:
1. Get the `y_pred` prediction on the `test set`:
2. Measure and show their MAE score for this model.
3. Measure and show their R2 scores for this model.

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score
# insert your code here


<details><summary><span style="color:red">&#x1F6C8; Help</span> (Use this only as a last resort!!)</summary>

```Python
y_pred = my_model.predict(X_test)
print(f"Test: mean_absolute_error: {mean_absolute_error( ... , y_pred):,.2f}")
print(f"Test: r2_score: {r2_score(  ...  , y_pred):,.2f}")
```

</details>

## Q5: - Measure the MAE and R2 errors for the `train` set of this new model:
1. Get the `y_pred` prediction  on the `train set`:
2. Measure and show their MAE score for this model.
3. Measure and show their R2 scores for this model.

In [ ]:
# insert your code here


<details><summary><span style="color:red">&#x1F6C8; Help</span> (Use this only as a last resort!!)</summary>

```Python
print('Similar to above, but you will use `X_train` and `y_train`.')
print('This is because we will use on the `training set`, instead of the `test set`.')
```

</details>

# Making a Learning curve plot:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def _plot_learning_curve(estimator, train_sizes, train_scores, valid_scores, metric='MAE', plt_text='', ax=None):
    train_errors = -train_scores.mean(axis=1)
    valid_errors = -valid_scores.mean(axis=1)
    ax = plt.gca() if not ax else ax
    ax.plot(train_sizes, train_errors, "r-+", linewidth=2, label="train")
    ax.plot(train_sizes, valid_errors, "b-", linewidth=3, label="valid")
    ax.set_xlabel("Training set size")
    ax.set_ylabel(f'{metric}')
    # plt.gca().set_xscale("log", nonpositive='clip')
    ax.grid()
    ax.legend(loc="upper right")
    ax.set_ylim(bottom=0, top=1.25*max([max(train_errors), max(valid_errors)]))
    ax.set_title(f'{estimator.__class__.__name__}\n{plt_text}', fontsize=8)


## Q6: - Show the learning curve plots of this model:
1. Using the function provided,  create the `learning curve` Plots for your new model.

*NB: As your new model is (probaly) more complex than the LinearRegression, it can take longer to perform this function. If so, reduce the 10 and 5 values to smaller numbers.*

In [ ]:
from sklearn.model_selection import learning_curve
# insert your code here.



<details><summary><span style="color:#17c1e5">&#x1F6C8; Hint</span> (Extra info)</summary>

* First call the `.. = learning_curve(..)` function. Examples in the Tutorial.ipynb.
* Second use the plot function above.
```Python
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 3), sharey=True, sharex=True)
_plot_learning_curve(my_model, train_sizes, train_scores, valid_scores, metric='MAE', plt_text='', ax=None)
```

</details>

<details><summary><span style="color:red">&#x1F6C8; Help</span> (Use this only as a last resort!!)</summary>

```Python
train_sizes, train_scores, valid_scores = learning_curve( my_model ,
                    X_train , y_train ,
                    train_sizes=np.linspace(0.05, 1.0, 10), # 10 size intervals, from 5% to 100%
                    cv=5,     # CV=5 means  Train = 80%  , Valid = 20%.
                              # CV=10 means Train = 90%  , Valid = 10%.
                              #   - The fit/predict is repeated 5 times with random samples taken from X/Y.
                              #   - The resulting error is the average across all 5 trials; so a smoother and fairer result than CV=1 , which is hold-out.
                    scoring="neg_mean_absolute_error"
                )
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 3), sharey=True, sharex=True)
_plot_learning_curve(my_model, train_sizes, train_scores, valid_scores, metric='MAE', plt_text='', ax=None)
```

</details>

## Q7: - Write an explanation for your model's learning curve:

1. According to the learning curve, is the model `overfitted` or `underfitted` or has a `generalized fit`? Explain your decision.

Your answer: 
- `[type your answer]`

<details><summary><span style="color:red">&#x1F6C8; Help</span> (Use this only as a last resort!!)</summary>


> Refer to the slides for pictures of different fits.

> * Most model algorithms will give a `"GENERALIZED FIT"` by default. i.e. both train and valid error scores are close together.
> * If train and valid error scores are close together, but the MAE error is high (like 300,000 Dollars!), then we might consider this `"UNDERFIT"`. - Underfitting both training and testing data.
> * If Test/Valid Error is high, but Train Error is low, then this is `"OVERFIT"` - overfitting the training data.
> 
> The difference between `GENERALIZED FIT` and `UNDERFIT`, might depend on what is a good error level.
> 
> For example:
> * We might decide that 50,000 AUD is good (MAE) error.
>    * If MAE is > 50,000 AUD, then we might say the model is still `"UNDERFITTING"`.
> 
> * Or, we might decide that R2 > 0.8 is good.
>    * If R2 is < 0.8, then perhaps the model is still `"UNDERFITTING"`.
> 
> You can decide what than you would like (r2=1.0 = 100% is perfect! as is MAE = $0 dollars). There's a human decision here to make :-)
> 
> If `UNDERFITTING`, we might want to take more model improvement steps: including more features, feature representations or a more complex model.
> 

</details>

## Q8: - Compare Models: - Write an explanation for your model's errors:

Answer the following:

From Lab_07_Tutorial and above, find and fill-in the following information for the `Test Set`:

1. `Polynomial Degree-2` model's `MAE Score` was: ________
2. `Polynomial Degree-2` model's `R2 Score` was: ________
3. `My_Model` model's `MAE Score` was: ________
4. `My_Model` model's `R2 Score` was: ________
5. Is the `MAE error` and the `R2 score` better or worse?
    - Is `My_Model` Better / Worse?:  _________
    - What is the `MAE (AUD $)` difference between models?:  _________
    - What is the `R2 Score` difference between models?:  _________

<details><summary><span style="color:red">&#x1F6C8; Help</span> (Use this only as a last resort!!)</summary>

```
Check your solutions above and Lab07_Tutorial for specific answers.
The Polynomial Degree-2 model's:
- MAE Score is ~ 323,713.60.
- R2 Score is ~ 0.43.
The My_Model model's:
- MAE Score is ~ ????????
- R2 Score is ~ ???
```

</details>

# Export your model to a file:


In [ ]:
import sys, os
import pickle

def export_model_to_file( model ):
    print('Saving files to directory: ', os.getcwd())
    model_name = f'x_Exported_Model_housing-model'
    ofn1 = f'{model_name}.pkl'
    with open(ofn1, 'wb') as file:
        pickle.dump(model, file)
        print(f'Exported to file:\t {ofn1}')
    model_bytes = pickle.dumps(model)
    model_str = model_bytes.decode('latin1')
    ofn2 = f'{model_name}.txt'
    with open(ofn2, 'w') as file:
        file.write( '\n'.join( [
            f'The {model.__class__.__name__} Model is {len(model_str):,} chars in total, at (approx.) memory size: {sys.getsizeof(model_bytes)/1024:,.2f} KiB\n\n', 
        ]))
        print(f'Exported to file:\t {ofn2}')

## Q9: - Export your trained model as a file:

In [ ]:
# insert your code here.



# Steps Run the Streamlit Dashboard from a new Anaconda Terminal. Make a Prediction.

1. Open a new Anaconda terminal.
2. Browse to your `Lab Package directory` (as usual).
3. You should find your two (2) newly exported files containing your model (.pkl).
4. Type `cd dash` to enter into the Dashboard directory.
5. If you do not not already have streamlit installed, then you can now `!pip install streamlit` (top of this file).
6. In your terminal, type: `streamlit run app.py` this should run the web-based dashboard with your model.
    * The command should open your default web browser.
    * It should open the page at the URL for your dashboard on your localhost, e.g. http://localhost:8501
    * If not, check the output messages for errors.
    * *NB: Ctrl+C on the terminal will close the dashboard web service.*
7. Make a `new prediction` at the bottom of the dashboard page.
8. Open the `dash/predictions_data.csv` file in Excel to confirm your prediction and input was stored.

## Q10: Show your Prediction & Explain The Error of your Model to your LA
1. Show `your input values` and  `your model's predicted value` *(Insert a screenshot if you like)*
2. Show your saved prediction in `dash/predictions_data.csv` *(Insert a screenshot if you like)*
3. Tell your LA - `how much average MAE error ($-AUD)` and `R2 score` is `attached to your model and its prediction`. (See your Q4). 

```






```
<p style="text-align:center;">That's it! Congratulations! <br> 
    Now, call an LA to check your solution. Then, upload your code on MyCourses.</p>

```














```
## **OPTIONAL EXTRA** - Partial Dependency Plots:
### Intepreting Features on Price for our new model:

- Like the `Residual Error Plots per Feature` we saw in the Tutorial, `Partial Dependency Plots` allow us see the average (independent) feature effect on `Y` in relative terms (i.e. relative house price change).

Key points:

> - PDP are to show an average of the marginal effects of a feature and  express the behaviour of the trained predictive model. [Hastie, 2009]
> - The **primary assumption** for interpreting PDPs is that the features are independent (i.e. not influencing one-another).
> - Note, this information of feature effects **cannot** be interpreted as effects in the underlying data or in the true population's behaviour. This is model interpretation, having parsed the dataset. That is different from Data Analysis, because the model has error: `f(x,p) = Y + error + noise`. Findings can be drawn from data analysis, specifically correlations (on the data sample).
> 
> Reference: [H2009] T. Hastie, R. Tibshirani and J. Friedman, The Elements of Statistical Learning, Second Edition, Section 10.13.2, Springer, 2009.

- Run the code below.

In [ ]:
from sklearn.inspection import partial_dependence, PartialDependenceDisplay
import matplotlib.pyplot as plt

# features = ...

fig, ax = plt.subplots(ncols=len(features), sharey=True, figsize=(16,4))
_ = [a.axhline(y=0, color='r', linestyle='dashed') for a in ax] if isinstance(ax, np.ndarray) else ax.axhline(y=0, color='r', linestyle='dashed')
PartialDependenceDisplay.from_estimator(my_model, 
                                   features=list(range(len(features))), 
                                   X=X_test, 
                                   feature_names=features, 
                                   # grid_resolution=10,
                                   ax=ax
                                       )

### Analysis of plots:
- The blue line reports the effect of each variable's value on `Y`, which is the property/house `Price`.
- The red dashed-line shows the intercept (crossing-point) at which the variable has no effect on the predicted `Y` value.
- Falling and Rising trends:
    - Falling, suggests a negative correlation with `Y`.
    - Rising, suggests a positive correlation with `Y`.
    - We can test this by normalizing the two variables and applying Pearson's Correlation Co-efficient (R) test. A rough version of that is below. The partial dependency are therefore consistent with the notion of statistical correlation.
    - Here:
        - We are analysing the `prediction model's representation of the true population`, as learnt from the dataset.
        - We are not analysing the data sample of the true population (as we might do in Statistical Data Analysis).

In [ ]:
from scipy.stats import pearsonr

def pearsons_r(a,b, alpha=0.05):
    corr_coef, p_value = pearsonr(a,b)
    print(f"Pearson correlation coefficient: {corr_coef}")
    print(f"P-value: {p_value}")
    print('Significantly' if p_value<alpha else 'Not significantly', 'Falling' if corr_coef<0 else 'Rising' if corr_coef>0 else 'Equal')
dft = df.copy().dropna()

In [ ]:
pearsons_r( dft['Distance'] , dft['Price'] )

In [ ]:
pearsons_r( dft['BuildingArea'] , dft['Price'] )